## **IMPORT LIBRARIES**

In [1]:
import numpy as np
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import Audio

## **AUDIO PLOTTER**

In [2]:
def plot_overlay_waveform(y_input, y_denoised, sr, title="Audio Comparison"):

    # Ensure same length
    min_len = min(len(y_input), len(y_denoised))
    y_input = y_input[:min_len]
    y_denoised = y_denoised[:min_len]

    # Time axis
    t = np.arange(min_len) / sr

    # Plot overlay
    plt.figure(figsize=(12, 4))
    plt.plot(t, y_input, alpha=0.6, label="Input (Noisy)", color="red")
    plt.plot(t, y_denoised, alpha=0.8, label="Denoised", color="blue")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

ALL NEW ADDITIONS GO BELOW HERE

## **WIENER FILTERING IMPLEMENTATION FOR STATIONARY NOISE**

In [3]:
def wiener_denoise(
    input_path: str,
    output_path: str,
    noise_duration: float = 0.5,
    n_fft: int = 1024,
    hop_length: int = 512
):

    y_noisy, sr = librosa.load(input_path, sr=None)

    D = librosa.stft(y_noisy, n_fft=n_fft, hop_length=hop_length)
    magnitude, phase = np.abs(D), np.angle(D)

    noise_frames = magnitude[:, :int(noise_duration * sr / hop_length)]
    noise_power = np.mean(noise_frames**2, axis=1, keepdims=True)

    signal_power = magnitude**2
    alpha = 12
    H = signal_power / (signal_power + (alpha * noise_power) + 1e-12)  # avoid divison by zero
    D_denoised = H * D

    y_denoised = librosa.istft(D_denoised, hop_length=hop_length)

    sf.write(output_path, y_denoised, sr)
    print(f"[INFO] Denoised audio saved at: {output_path} with alpha = {alpha}")

    return y_denoised, sr

## **PERFORM WIENER FILTERING**

In [ ]:
from google.colab import files

uploaded = files.upload()
input_path = list(uploaded.keys())[0]

y_denoised, sr = wiener_denoise(
    input_path,
    output_path="denoised_wiener_audio.wav"
)

In [ ]:
Audio("Fan_Noise.wav") #LISTEN TO NOISY AUDIO

In [ ]:
Audio("denoised_wiener_audio.wav") #LISTEN TO DENOISED AUDIO